In [1]:
import sys
#BASE_DIR = "/home/dzigen/Desktop/ITMO/smiles2024/RAG-project-SMILES-2024-"
BASE_DIR = "/trinity/home/team06/workspace/mikhail_workspace/rag_project"
sys.path.insert(0, BASE_DIR)

In [2]:
from src.Scorer import LLM_UncertaintyScorer, UncertaintyScorerConfig
from src.Reader import LLM_Model, LLM_Config

/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
reader_config = LLM_Config()
READER = LLM_Model(reader_config)

In [3]:

scorer_config = UncertaintyScorerConfig(
        system_prompt = '''Ты - экспертная система, задача которой - оценить релевантность документа по отношению к данному вопросу. На вход поступает вопрос и документ в текстовом формате, твоя задача - вернуть одну из двух меток: "{cert_label}" - в документе содержится информация, которая может быть использована для верного ответа на данный вопрос и "{uncert_label}" - в документе не содержится информации, которая может быть использована для верного ответа на данный вопрос. Не пиши ничего другого, кроме предсказываемой метки релевантности: "{cert_label}" либо "{uncert_label}".

Примеры: 

Вопрос: Кто такой барсук?
Документ: Барсу́к, или обыкнове́нный барсу́к (лат. Meles meles) — вид млекопитающих из рода барсуков семейства куньих.
Метка: {cert_label}

Вопрос: Где обитают дельфины?
Документ: Розетки, рассчитанные на 220 Вольт, обычно используют для подключения различных электрических приборов, мощность которых не превышает 3,5 кВт.
Метка: {uncert_label}

Вопрос: Когда родился Александр Сергеевич Пушкин?
Документ: Ива́н Серге́евич Турге́нев (28 октября (9 ноября) 1818, Орёл, Российская империя — 22 августа (3 сентября) 1883, Буживаль, Франция) — русский писатель-реалист, поэт, публицист, драматург, переводчик.
Метка: {uncert_label}''',
    user_prompt_template = "Вопрос: {q}\nДокумент: {c}\nМетка: "
)

In [5]:
SCORER = LLM_UncertaintyScorer(scorer_config, READER)

In [6]:
examples = [
    []
    ["Когда родился Александр Сергеевич Пушкин?", "Ф. М. Достоевский родился 30 октября (11 ноября) 1821 г. в семье врача московской Мариинской больницы для бедных Михаила Андреевича Достоевского."],
    ["Кто написал книгу 'Атлант расправил плечи'?", "«Атла́нт расправил плечи» (англ. Atlas Shrugged, дословно — «Атлант пожал плечами») — роман-антиутопия американской писательницы Айн Рэнд, впервые опубликованный в 1957 году. Является четвёртым и последним романом Рэнд, а также самым длинным."]
]

In [8]:
for i, example_pair in enumerate(examples):
    print("\nEXAMPLE #1")
    print(f"QUERY:\n{example_pair[0]}")
    print(f"CONTEXT:\n{example_pair[1]}")
    
    predicted_score = SCORER.predict(example_pair[0], [example_pair[1]])
    print(f"SCORE:\n{predicted_score[0]}")


EXAMPLE #1
QUERY:
Когда родился Александр Сергеевич Пушкин?
CONTEXT:
Ф. М. Достоевский родился 30 октября (11 ноября) 1821 г. в семье врача московской Мариинской больницы для бедных Михаила Андреевича Достоевского.
SCORE:
нерелевантный

EXAMPLE #1
QUERY:
Кто написал книгу 'Атлант расправил плечи'?
CONTEXT:
«Атла́нт расправил плечи» (англ. Atlas Shrugged, дословно — «Атлант пожал плечами») — роман-антиутопия американской писательницы Айн Рэнд, впервые опубликованный в 1957 году. Является четвёртым и последним романом Рэнд, а также самым длинным.
SCORE:
релевантный
